In [25]:
import csv
from openai import OpenAI
import os
import re

# OPENAI_API_KEYを含むとPushできないため実行時は有効にしてください
# os.environ["OPENAI_API_KEY"] = "*****"

# openAIの機能をclientに代入
client = OpenAI()

# 対象者のMBTIタイプ
user_mbti = "INTJ"

In [26]:
# CSVファイルを読み込み
with open('../output.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    people = list(reader)

In [30]:
def find_best_matches(people, user_mbti, top_n=3):
    prompts = []
    persons = ""
    for person in people:
        persons = persons + str(f"【名前】 {person['Slack表示名']}, 【自己紹介】 {person['自己紹介']}, 【得意な業界】 {person['業界']}, 【関心のある領域】 {person['関心のある領域']}, 【協働で大切にしたいこと】 {person['Tech0のPJTで8期の仲間と協働する際に大切にしたいと思うこと']}, 【プロジェクト推進で得意なこと/苦手なこと】 {person['PJTをする上で自分が得意なこと・苦手なこと']}, 【参加動機や達成したいこと】 {person['Tech0の参加動機と１年後に到達したい・達成したいこと']},") + "\n"
    prompt = f"MBTIタイプが{user_mbti}の人物と合う人物を以下から3名選んでください。対象者は、名前：,選定理由:形式で出力してください\n " + persons


    response =  client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": prompt },
        ],
    )
        
    # コンテンツを抽出
    content = response.choices[0].message.content.strip()
    
    # 対象者と選定理由を抽出するための正規表現パターン
    pattern = r"\*\*名前\*\*: ([\w\s-]+)\s+\*\*選定理由\*\*: ([\s\S]+?)(?=\n\d|$)"
    # 正規表現による抽出
    matches = re.findall(pattern, content)
    # 結果を配列に格納
    result = [{"名前": match[0], "選定理由": match[1].strip()} for match in matches]

    return result

In [31]:
# 最適な3人を選択
best_matches = find_best_matches(people, user_mbti)

# 結果を表示
for match in best_matches:
     print(match)

以下の3名を選びました。

1. **名前**: Takahashi Hiroaki-8  
   **選定理由**: 新規事業や起業に興味を持ち、目標に向かって具体的なスキルを身に着けようとする姿勢がINTJの特性に合っています。また、自己成長を重視し、成功する秘訣を分かち合いたいという理念もINTJに共鳴します。

2. **名前**: Sugiyama Shoichi-8  
   **選定理由**: 複数のバックグラウンドを持ちながら、テクノロジーとのコミュニケーションを図る姿勢が、INTJのロジカルで戦略的な思考に合致しています。また、新規事業開発に従事し、適度な距離感を大事にする姿勢がINTJの特性にマッチします。

3. **名前**: Miyazawa Yoshiki-8  
   **選定理由**: SaaSプロダクトのPdMとしての経験を持ちつつ、ビジネス領域を超えたTechスキルの習得を目指す姿勢がINTJの行動特性に合っています。分析的思考と戦略的ビジョンを持ち合わせたこの人物は、INTJに共通する特性を体現しています。
[{'名前': 'Takahashi Hiroaki-8  \n  ', '選定理由': '新規事業や起業に興味を持ち、目標に向かって具体的なスキルを身に着けようとする姿勢がINTJの特性に合っています。また、自己成長を重視し、成功する秘訣を分かち合いたいという理念もINTJに共鳴します。'}, {'名前': 'Sugiyama Shoichi-8  \n  ', '選定理由': '複数のバックグラウンドを持ちながら、テクノロジーとのコミュニケーションを図る姿勢が、INTJのロジカルで戦略的な思考に合致しています。また、新規事業開発に従事し、適度な距離感を大事にする姿勢がINTJの特性にマッチします。'}, {'名前': 'Miyazawa Yoshiki-8  \n  ', '選定理由': 'SaaSプロダクトのPdMとしての経験を持ちつつ、ビジネス領域を超えたTechスキルの習得を目指す姿勢がINTJの行動特性に合っています。分析的思考と戦略的ビジョンを持ち合わせたこの人物は、INTJに共通する特性を体現しています。'}]
{'名前': 'Takahashi Hiroaki-8  \n  ', '選定理由': '新規事業